## Teste de PostgrSQL



In [32]:
import psycopg2

conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost"
)

cur = conn.cursor()
cur.execute("SELECT * FROM cdmdatabaseschema.person_details LIMIT 5;")
print(cur.fetchall())

cur.close()
conn.close()

[]


### Carregar Ficheiro

In [33]:
import pandas as pd


file_path = "detectid.csv"
df = pd.read_csv(file_path)


df.head()

,Pessoa,Tempo,Dia de Medição,Hora de Medição,Data de Nascimento,Genero,NIVEL DE CONSCIÊNCIA,FREQUÊNCIA CARDIACA,TA Sistólica,TA Diastólica,TEMPERATURA,SpO2,NECESSIDADE DE O2,DOR,DESCOMPENSAÇÃO,Ativação Médico,Aumento da Vigilância,Via Área Ameaçada,Evento,Area\nMonotorizada
0,Pessoa 1,0.0,11/11/2023,11:00,04/08/1965,Masculino,15,103.0,109.0,61.0,37.1,94.0,1.0,0.0,0.0,0.0,0.0,0.0,0,NaN
1,Pessoa 1,125.5,16/11/2023,18:30,04/08/1965,Masculino,15,99.0,110.0,74.0,37.1,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,Pessoa 1,127.5,16/11/2023,20:30,04/08/1965,Masculino,15,78.0,102.0,78.0,NaN,NaN,0.0,0.0,1.0,1.0,1.0,0.0,1,1.0
3,Pessoa 1,133.0,17/11/2023,02:00,04/08/1965,Masculino,15,108.0,87.0,53.0,36.8,96.0,0.0,0.0,1.0,0.0,1.0,0.0,1,1.0
4,Pessoa 1,138.5,17/11/2023,7:30,04/08/1965,Masculino,15,96.0,94.0,58.0,37.1,NaN,0.0,0.0,1.0,0.0,1.0,0.0,1,1.0


### Tratemento dos Dados

Aqui chatgpt para tratamento dos dados -> Rever isto tudo

In [35]:
# Tempo tem que ser calculado logo nao entra na base de dados
# Evento foi uma experiencia, tambem nao entra
df_cleaned = df.drop(columns=["Tempo", "Evento"])

df_cleaned.rename(columns=lambda x: x.strip().replace("\n", " "), inplace=True)

# Valores Omissos preenchidos com média
numeric_cols = df_cleaned.select_dtypes(include='number').columns
df_cleaned[numeric_cols] = df_cleaned[numeric_cols].fillna(df_cleaned[numeric_cols].mean())

# Converter para numérico (caso haja strings ou espaços)
df_cleaned["NIVEL DE CONSCIÊNCIA"] = pd.to_numeric(df_cleaned["NIVEL DE CONSCIÊNCIA"], errors='coerce')

# Calcular a média ignorando os NaNs e arredondar para inteiro
media_nivel = int(df_cleaned["NIVEL DE CONSCIÊNCIA"].mean().round())

# Preencher os NaNs com a média
df_cleaned["NIVEL DE CONSCIÊNCIA"].fillna(media_nivel, inplace=True)

# Converter a coluna para datetime (assumindo formato MM/DD/YYYY)
df_cleaned["Data de Nascimento"] = pd.to_datetime(df_cleaned["Data de Nascimento"], format="%m/%d/%Y", errors="coerce")

# Criar colunas separadas para ano, mês e dia
df_cleaned["Ano de Nascimento"] = df_cleaned["Data de Nascimento"].dt.year
df_cleaned["Mes de Nascimento"] = df_cleaned["Data de Nascimento"].dt.month
df_cleaned["Dia de Nascimento"] = df_cleaned["Data de Nascimento"].dt.day

# Garantir que 'Dia' e 'Hora' são string
df_cleaned["Dia de Medição"] = df_cleaned["Dia de Medição"].astype(str)
df_cleaned["Hora de Medição"] = df_cleaned["Hora de Medição"].astype(str)

# Juntar as colunas 'Dia' e 'Hora' e converter para datetime com formato automático
df_cleaned["datetime"] = pd.to_datetime(df_cleaned["Dia de Medição"] + " " + df_cleaned["Hora de Medição"], dayfirst=True, errors="coerce")

df_cleaned["person_id"] = df_cleaned["Pessoa"].str.extract(r"(\d+)").astype(int)

# Ver exemplo de dados limpos e prontos para inserção
print(df_cleaned.to_string())


         Pessoa Dia de Medição Hora de Medição Data de Nascimento      Genero  NIVEL DE CONSCIÊNCIA  FREQUÊNCIA CARDIACA  TA Sistólica  TA Diastólica  TEMPERATURA        SpO2  NECESSIDADE DE O2       DOR  DESCOMPENSAÇÃO  Ativação Médico  Aumento da Vigilância  Via Área Ameaçada  Area Monotorizada  Ano de Nascimento  Mes de Nascimento  Dia de Nascimento            datetime  person_id
0      Pessoa 1     11/11/2023           11:00         1965-04-08   Masculino                  15.0           103.000000    109.000000      61.000000    37.100000   94.000000           1.000000  0.000000        0.000000         0.000000               0.000000               0.00           0.088889               1965                  4                  8 2023-11-11 11:00:00          1
1      Pessoa 1     16/11/2023           18:30         1965-04-08   Masculino                  15.0            99.000000    110.000000      74.000000    37.100000   96.000000           0.000000  0.000000        0.000000         

C:\Users\gonca\AppData\Local\Temp\ipykernel_12384\3459433511.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned["NIVEL DE CONSCIÊNCIA"].fillna(media_nivel, inplace=True)


### Inserção dos dados PERSON

In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

# Conectar à base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

rows = []
person_id = 0
for _, row in df_cleaned.iterrows():
        if person_id < row["person_id"]:
            person_id = row["person_id"]
            genero = 1 if row['Genero'] == 'Masculino' else 0
            rows.append((
                person_id,
                genero,
                row["Ano de Nascimento"],
                row["Mes de Nascimento"],
                row["Dia de Nascimento"],
                "12345555"
            ))

query = """
INSERT INTO cdmDatabaseSchema.PERSON (
    person_id, gender_concept_id, year_of_birth, month_of_birth, day_of_birth, person_source_value
) VALUES %s
"""

execute_values(cur, query, rows)
conn.commit()

print("Dados PERSON inseridos com sucesso!")

cur.close()
conn.close()


Dados PERSON inseridos com sucesso!


### Inserção dos Dados MEASUREMENT

In [36]:
# Conectar à base de dados
conn = psycopg2.connect(
    dbname="detectid",
    user="postgres",
    password="Goncalo123",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

# Exemplo de inserção na tabela MEASUREMENT
measurement_concepts = {
    "TA Sistólica": 1,
    "TA Diastólica": 2,
    "TEMPERATURA": 3,
    "SpO2": 4,
    "DOR": 5,
    "NECESSIDADE DE O2": 6,
    "FREQUÊNCIA CARDIACA": 7,
    "NIVEL DE CONSCIÊNCIA": 8
}

measurement_id = 1
rows = []
for _, row in df_cleaned.iterrows():
    for field, concept_id in measurement_concepts.items():
        rows.append((
            measurement_id,
            int(row["person_id"]),
            concept_id,
            row[field],
            row["datetime"]
        ))
        measurement_id += 1

query = """
INSERT INTO cdmDatabaseSchema.MEASUREMENT (
    measurement_id, person_id, measurement_concept_id, value_as_number, measurement_datetime
) VALUES %s
"""


execute_values(cur, query, rows)
conn.commit()

print("Dados MEASUREMENT inseridos com sucesso!")
cur.close()
conn.close()


Dados MEASUREMENT inseridos com sucesso!


### Inserção dos Dados CARE_SITE

### Inserção dos Dados CONDITION_OCCURENCE

### Inserção dos Dados NOTE

### Inserção dos Dados OBSERVATION

### Inserção dos Dados PERSON_DETAILS

### Inserção dos Dados VISIT_OCCURENCE

### Extração dos Dados da BD para Dataset